# Calibration of the standard consumption-saving model

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Calibration of aggregate wealth](#toc2_)    
- 3. [Het-$\beta$ model](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

**Content:** Solves and simulates a simple consumption-saving model.

**Calibration:** Low and high liquid wealth calibrations.

**Model extension:** Discount factor heterogeneity

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

import scipy.optimize as opt
from ConSavModel import ConSavModelClass

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = ConSavModelClass()

## 2. <a id='toc2_'></a>[Calibration of aggregate wealth](#toc0_)

In [3]:
def calibrate_agg_wealth(x, model, wealth_to_income_target):
    
    # a. guess on beta 
    model.par.beta = x 

    # b. solve
    model.solve(do_print=False)

    # c. simulate
    model.simulate(do_print=False)

    # d. aggregate 
    A = np.sum(model.sim.D*model.sol.a)
   
    return wealth_to_income_target - A/model.par.w 

Calibrate model with high level of liquidity 

In [4]:
# Wealth target 
target_high_wealth = 4.1 * 4 # annual wealth to income ratio of 4.1, converted to quarterly frequency  

### ADD CODE HERE ###
# Add root finder (using the scipy package for instance) which findes the root of calibrate_agg_wealth() 

MPCs 

In [ ]:
def get_mpcs(model, do_print=False):
    c = model.sol.c 
    a = model.sol.a  
    par = model.par
    a_grid = par.a_grid 

    # Initialize and post return
    mpc = np.zeros_like(c)
    post_return = (1 + par.r) * a_grid

    # Calculate mpc for the inner part of the array as the slope of the consumption function 
    mpc[:,:, 1:] = np.diff(c, axis=2) / np.diff(post_return)

    # Handle boundary cases (first and last columns)
    mpc[:,:, 0] = (c[:,:, 1] - c[:,:, 0]) / (post_return[1] - post_return[0])
    mpc[:,:, -1] = (c[:,:, -1] - c[:,:, -2]) / (post_return[-1] - post_return[-2])

    # Set MPC to 1 if constrained 
    mpc[a == a_grid[0]] = 1

    MPC = np.sum(mpc*model.sim.D)
    if do_print: print(f'mean MPC: {MPC:6.3f}')

    return mpc

_ = get_mpcs(model, do_print=True)

Plot MPCs and distribution 

In [ ]:
def mpc_plot(model, a_max=5., N_bins=6):

    # mpc 
    mpc = get_mpcs(model, do_print=False)

    # unload 
    par = model.par 
    D = model.sim.D 
    z_ergodic = par.z_ergodic
    a_grid = par.a_grid
    mpc_a = np.sum(mpc*z_ergodic[None,:,None], axis=(0,1))  # average over income in order to plot over assets only 
    D_a = np.sum(D, axis=(0,1))

    # plot settings for histogram 
    i_a_max = np.searchsorted(a_grid, a_max) - 1

    # bin distribution for better plot 
    from scipy.stats import binned_statistic
    D_A_binned, bin_edges, binnumber = binned_statistic(a_grid[:i_a_max], D_a[:i_a_max], statistic='sum', bins=N_bins)
    D_A_binned[-1] += np.sum(D_a[i_a_max:]) # add all mass above a_max to top bin 
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2 # get bin centers for plot 

    # Plot 
    fig, ax1 = plt.subplots()   

    # Plot mpc
    ax1.plot(a_grid[:i_a_max], mpc_a[:i_a_max], color='black', label='mpc', linewidth=3.5)
    ax1.set_xlabel('Assets')
    ax1.set_ylabel('mpc')
    ax1.tick_params(axis='y')
    ax1.set_ylim([0.,0.7])

    # Create a second y-axis for the histogram
    ax2 = ax1.twinx()
    ax2.bar(bin_centers, D_A_binned, width=bin_edges[1] - bin_edges[0], color='royalblue', alpha=0.6, label="Distribution")
    ax2.set_ylabel('Mass')
    ax2.tick_params(axis='y')
    ax2.set_ylim([0.,1.])

    plt.title("Plot of MPCs and distribution D")
    plt.show()

mpc_plot(model)

Calibrate model with low level of liquidity 

In [7]:
model_low_wealth = model.copy()
target_low_wealth = 0.5 * 4

### ADD CODE HERE ###

Aggregate MPC 

In [ ]:
_ = get_mpcs(model_low_wealth, do_print=True)

plot mpcs 

In [ ]:
mpc_plot(model_low_wealth)

## 3. <a id='toc3_'></a>[Het-$\beta$ model](#toc0_)

Modify the household problem to allow for different discount factors across the population. In particular, the model is given by:
\begin{aligned}V\left(a_{t-1},z_{t},\beta'\right)= & \max_{c_{t}}u\left(c_{t}\right)+\beta'\mathbb{E}\left[V\left(a_{t},z_{t+1},\beta'\right)\right]\\
 & \text{ subject to }\\
 & c_{t}+a_{t}\leq a_{t-1}\left(1+r\right)+z_{t}\\
 & a_{t}\geq\underline{a}
\end{aligned}
where $\beta$ comes from an evenly spaced grid with $N_\beta$ points between $\beta-2\Delta,\beta+2\Delta$. 
Calibrate the model to match both a high level of liquidity and a high aggregate MPC using $(\beta, \Delta)$.

Note: You will need to update the code to set the beta grid (see next cell), and update the household problem (*solve_hh_backwards_egm*) appropriately.

In [10]:
model_hetbeta = ConSavModelClass()
model_hetbeta.par.Nbeta = 2 # Number of beta grid points 
model_hetbeta.allocate() # update size of beta grid 
model_hetbeta.par.beta_grid[:] = ... # Values in grid 

In [ ]:
model_hetbeta.solve(do_print=True)

In [ ]:
model_hetbeta.simulate(do_print=True)

Check the distribution 

In [ ]:
print('Mass of distribution (should be 1):', np.sum(model_hetbeta.sim.D))
assert np.isclose(np.sum(model_hetbeta.sim.D), 1.0 )

Check how many households are in the beta states. If the distribution is uniform it should have an equal number of households across the beta grid

In [ ]:
beta_mass = np.sum(model_hetbeta.sim.D,axis=(1,2)).round(2)
print(f'Mass of households across beta grid: {beta_mass}')
print(f'Should be {1/model_hetbeta.par.Nbeta}')

Next, calibrate the model to both feature a high MPC and a high level of wealth

In [15]:
### CODE HERE ###

Check MPC

In [ ]:
_ = get_mpcs(model_hetbeta, do_print=True)

Plot MPCs  

In [ ]:
mpc_plot(model_hetbeta)

*Final question*. Try changing the mass of agents across the beta distribution. Set Nbeta=3 and assume that the mass is distributed as [0.2, 0.6, 0.2] across the beta grid. What part of the code do you have to modify?